<a href="https://colab.research.google.com/github/YoungGaLee/AI_Study/blob/master/%5B%EC%8B%A4%EC%8A%B5_2%5D%20Brain_classification/%EB%AC%BC%EC%9D%80_%EB%AC%BC%EC%9D%B4%EC%9A%94_%EC%82%B0%EC%9D%80_%EC%82%B0%EC%9D%B4%EB%A1%9C%EB%8B%A4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()  # 파일 업로드 버튼 생성


Saving 인공지능 두뇌지수 데이터_1024.xlsx to 인공지능 두뇌지수 데이터_1024.xlsx


In [2]:
import numpy as np
import torch
import torch.nn as nn #nn.Linear를 쓰기 위해
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

torch.manual_seed(1)

In [0]:
df = pd.read_excel('인공지능 두뇌지수 데이터_1024.xlsx',sep=',', header=0, usecols=[1,2,3,4,5,6,8,9,10,11,12,13,14],dtype=np.float32) 
#np.loadtxt로는 엑셀을 열기 어려우므로 pd.read_csv나 excel사용 왜 usecols를 쓰지 않으면 dtype = np.float32를 쓸 수 없을까?
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(df, test_size=0.001, random_state=0)
#data_train.shape, data_test.shape

In [0]:
train_data = np.array(data_train)
test_data = np.array(data_test)

x_train = torch.FloatTensor(train_data[:, 0:-1])
y_train = torch.LongTensor(train_data[:, [-1]]).squeeze()
#데이터 전처리 정규분포화

# mu = x_train.mean(dim=0)
# sigma = x_train.std(dim=0)
# norm_x_train = (x_train - mu) / sigma

x_test = torch.FloatTensor(test_data[:,0:-1])
y_test = torch.LongTensor(test_data[:, [-1]]).squeeze()


In [0]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(12, 26)
        #어째서 25이하는 안되는가?
    def forward(self, x):
        return self.linear(x)

In [0]:
model = SoftmaxClassifierModel()
# optimizer 설정
optimizer = optim.Adam(model.parameters(), lr=1e-4)
# 테스트 결과 optim.SGD보다 optim.Adam가 더 빠름. 
# https://medium.com/@Biboswan98/optim-adam-vs-optim-sgd-lets-dive-in-8dbf1890fbdc
# SGD는 학습 속도의 신중한 조정이 필요하지만 Adam에서는 그렇지 않습니다. 그러나 성과는 SGD 학습률보다 덜 민감합니다.

In [0]:
def train(model, optimizer, x_train, y_train):
  
  nb_epochs = 10000
  for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [19]:
train(model, optimizer, x_train, y_train)

Epoch    0/10000 Cost: 2599.797363
Epoch 1000/10000 Cost: 465.752899
Epoch 2000/10000 Cost: 133.744186
Epoch 3000/10000 Cost: 19.633715
Epoch 4000/10000 Cost: 5.407342
Epoch 5000/10000 Cost: 2.133953
Epoch 6000/10000 Cost: 1.265061
Epoch 7000/10000 Cost: 0.852436


KeyboardInterrupt: ignored

In [20]:
with torch.no_grad():

    prediction = model(x_test)
    prediction = prediction.argmax(-1)
    print(prediction, y_test)

tensor([10, 17, 14,  2,  3, 17, 24, 13, 10, 18,  9,  3,  1]) tensor([10, 17, 14,  2,  3, 17, 23, 13, 10, 18,  9,  3,  1])
